In [1]:
import bz2
import json
import numpy as np
import pandas as pd
import spacy
import pickle
from pathlib import Path

from IPython.display import display, HTML

from ressources import config

In [2]:
RAW_DATA_FOLDER = config.RAW_DATA_FOLDER
GENERATED_DATA_FOLDER = config.GENERATED_DATA_FOLDER

In [ ]:
def sanitarization(word_list):
    nlp = spacy.load("en_core_web_sm")
    word_list_san = []   
    for film in word_list:
        doc_film_name = nlp(film)
        tokens = [token.text for token in doc_film_name]
        result = " ".join(tokens)
        word_list_san.append(result)
    return word_list_san

## QUOTEBANK Dataset
Exploring and filtering of quotbank Dataset

In [3]:
QUOTEBANK_FOLDER = RAW_DATA_FOLDER / "QUOTEBANK"
file_list = list(QUOTEBANK_FOLDER.glob('*.json.bz2'))

### List of features

In [4]:
with pd.read_json(file_list[0], lines=True, compression='bz2', chunksize=1) as df_reader:
    for chunk in df_reader:
        df_quotebank = chunk
        break
# column list for Quotebank dataset
print(f"\nColumns quotebank:\n{df_quotebank.columns}")

IndexError: list index out of range

### Sample

In [5]:
# Sample for Quotebank dataset
print("\nSample quotebank:\n")
display(df_quotebank)


Sample quotebank:



NameError: name 'df_quotebank' is not defined

### Number of rows

In [ ]:
# Number of rows for quotebank dataset
chunksize = 10000
for file in file_list:
    n = 0
    print(f"Processing file {file.name}")
    with pd.read_json(file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            n += len(chunk)
            print(n, end = "\r")
    
    print(f"{n} rows in {file.name}")

Output:

Processing file quotes-2015.json.bz2</br>
20874338 rows in quotes-2015.json.bz2</br>
Processing file quotes-2016.json.bz2</br>
13862129 rows in quotes-2016.json.bz2</br>
Processing file quotes-2017.json.bz2</br>
26611588 rows in quotes-2017.json.bz2</br>
Processing file quotes-2018.json.bz2</br>
27228451 rows in quotes-2018.json.bz2</br>
Processing file quotes-2019.json.bz2</br>
21763302 rows in quotes-2019.json.bz2</br>

### Filtering out data
We decided to keep only the data that contains some keywords in the quote itself or in the url

In [ ]:
for file in file_list:
    path_to_out = GENERATED_DATA_FOLDER / "QUOTEBANK"
    path_to_out = path_to_out / f"{file.name.split('.', 1)[0]}-cinema.{file.name.split('.', 1)[1]}"
    with bz2.open(file, 'rb') as in_file:
        with bz2.open(path_to_out, 'wb') as out_file:
            for instance in in_file:
                instance = json.loads(instance)
                quote = instance['quotation']
                urls = instance['urls']
                if 'cinema' in quote or 'film' in quote or 'movie' in quote:
                    d_file.write((json.dumps(instance)+'\n').encode('utf-8'))
                elif:
                    for url in urls:
                        if 'cinema' in url or 'film' in url or 'movie' in url:
                            d_file.write((json.dumps(instance)+'\n').encode('utf-8'))
                            break

### Ouvrir quotebank 2018 et rajouter les colonnes et l'enregistrer 

In [12]:
#open wikidata dictionary
with open('../generated/WIKIDATA/dict_wikidata_people.pickle', 'rb') as f: 
     dict_wikidata_people = pickle.loads(f.read())

In [99]:
#teste pour faire la fonction en bas (a supprimer si plus besoins)
next(iter(dict_wikidata_people.values())) #acces to values
next(iter(dict_wikidata_people)) #acces to key
next(iter(dict_wikidata_people.items())) #acces to both
print(dict_wikidata_people.items())
print(dict_wikidata_people)
for q in dict_wikidata_people.items() :
   # print(q)
    print(q[0])
    print(q[1]['genderlabel'])
    print(q[1]['dob_std'])

dict_items([('Q23', {'label': 'George Washington', 'genderlabel': 'male', 'dob_std': '22.02.1732'}), ('Q42', {'label': 'Douglas Adams', 'genderlabel': 'male', 'dob_std': '11.03.1952'}), ('Q1868', {'label': 'Paul Otlet', 'genderlabel': 'male', 'dob_std': '23.08.1868'})])
{'Q23': {'label': 'George Washington', 'genderlabel': 'male', 'dob_std': '22.02.1732'}, 'Q42': {'label': 'Douglas Adams', 'genderlabel': 'male', 'dob_std': '11.03.1952'}, 'Q1868': {'label': 'Paul Otlet', 'genderlabel': 'male', 'dob_std': '23.08.1868'}}
Q23
male
22.02.1732
Q42
male
11.03.1952
Q1868
male
23.08.1868


In [ ]:
path_to_file = '.../quotes-2018.json.bz2'
path_to_out = '.../people_quotes-2018.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            qids = instance['qids'] # extracting list of qids
            gender = []
            dob = []
            for qid in qids:
                for qid_wiki in dict_wikidata_people.items() : 
                    if qid == qid_wiki[0] : 
                        gender = q[1]['genderlabel']
                        dob = q[1]['dob_std']
                        gender.append(gender)
                        dob.append(dob)
                    else :
                        gender.append('None')
                        dob.append('None')
            instance['gender'] = gender # updating the sample with gender
            instance['dob'] = dob # updating the sample with date of bith
            d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file